In [3]:
import os
import json
import random

import nltk
import numpy as np
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


nltk.download("punkt")
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional but helps wordnet work better in some languages

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jihad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jihad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jihad\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

**ChatbotModel – Neural Network for Intent Classification**

This model is a fully connected feedforward neural network designed for classifying user input into chatbot intent categories.  
It consists of two hidden layers with ReLU activations and dropout regularization, and an output layer producing logits.

- Input: Bag-of-words or vectorized user input
- Output: Logits for each intent class
- Loss function: CrossEntropyLoss (includes softmax internally)

In [4]:
class ChatbotModel(nn.Module):
    """
    A simple fully connected feedforward neural network for intent classification.

    Architecture:
        - Input layer
        - Two hidden layers (with ReLU + Dropout)
        - Output layer (raw logits for CrossEntropyLoss)

    Args:
        input_size (int): Number of input features (e.g., vocabulary size)
        output_size (int): Number of output classes (e.g., number of intent tags)
    """

    def __init__(self, input_size, output_size):
        super(ChatbotModel, self).__init__()

        # First fully connected layer: input_size → 128 hidden units
        self.fully_connected_1 = nn.Linear(input_size, 128)
        # Second fully connected layer: 128 → 64
        self.fully_connected_2 = nn.Linear(128, 64)
        # Output layer: 64 → number of intent classes
        self.fully_connected_3 = nn.Linear(64, output_size)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        """
        Defines the forward pass through the network.

        Args:
            x (Tensor): Input tensor of shape (batch_size, input_size)

        Returns:
            Tensor: Output logits of shape (batch_size, output_size)
        """
        x = self.relu(self.fully_connected_1(x))
        x = self.dropout(x)

        x = self.relu(self.fully_connected_2(x))
        x = self.dropout(x)

        x = self.fully_connected_3(x)  # No softmax here; use CrossEntropyLoss
        return x

**ChatbotAssistant Class – Handles Intents, Vocabulary & Dataset Preparation**

This class is responsible for:
- Loading and parsing the `intents.json` file
- Creating a vocabulary from the user patterns
- Associating patterns with intent tags
- Preparing training data (`X`, `y`) for the model
- Optionally handling special function mappings per intent (like launching code)

It is the logic brain behind the chatbot's "understanding" phase.

**Preprocessing Methods – Tokenization, Lemmatization & Bag-of-Words**

This section includes:
- Tokenizing and lemmatizing input patterns
- Creating the vocabulary
- Building bag-of-words vectors
- Parsing the full `intents.json` file to map patterns to tags

**Dataset Preparation – Features and Labels**

This method prepares the training dataset (`X`, `y`) by converting tokenized patterns into bag-of-words vectors and mapping each to its corresponding intent index.

**Training, Saving & Inference**

This block includes:
- `train_model`: trains the neural network on your dataset
- `save_model` / `load_model`: persists and reloads the trained model
- `process_message`: performs inference and returns a relevant chatbot response

In [5]:
class ChatbotAssistant:
    """
    A class to load, preprocess, train, and use a chatbot model based on an intents JSON file.

    Responsibilities:
        - Load and parse intents.
        - Tokenize and lemmatize input patterns.
        - Build vocabulary and tag list.
        - Create training data (X, y).
        - Train, save, and load a model.
        - Process user input to return a relevant response.

    Args:
        intents_file (str): Path to the JSON file defining intents.
    """

    def __init__(self, intents_path):
        self.model = None                          # Trained model
        self.intents_path = intents_path           # Path to intents JSON

        self.documents = []                        # List of (tokens, tag)
        self.vocabulary = []                       # Set of unique words
        self.intents = []                          # List of intent tags
        self.intents_responses = {}                # tag → responses

        self.X = None                              # Feature matrix
        self.y = None                              # Label vector

    @staticmethod
    def tokenize_and_lemmatize(text):
        """
        Tokenizes and lemmatizes a given text input using NLTK.

        Args:
            text (str): A raw input phrase.

        Returns:
            List[str]: Lowercased, lemmatized word tokens.
        """
        lemmatizer = nltk.WordNetLemmatizer()
        tokens = nltk.word_tokenize(text)
        return [lemmatizer.lemmatize(word.lower()) for word in tokens]

    def bag_of_words(self, tokenized_sentence):
        """
        Converts a tokenized sentence into a binary bag-of-words vector.

        Args:
            tokenized_sentence (List[str]): Preprocessed tokens.

        Returns:
            List[int]: A binary list representing presence/absence of vocab words.
        """
        return [1 if word in tokenized_sentence else 0 for word in self.vocabulary]

    def parse_intents(self):
        """
        Loads and processes the intents JSON file.
        If the same tag appears multiple times, assigns unique names by appending _1, _2, etc.
        """
        lemmatizer = nltk.WordNetLemmatizer()
        tag_counts = {}  # To track and rename duplicate tags

        if os.path.exists(self.intents_path):
            with open(self.intents_path, 'r', encoding='utf-8') as f:
                intents_data = json.load(f)

            for intent in intents_data['intents']:
                base_tag = intent['tag']
                # Create unique tag if it’s already been used
                if base_tag not in tag_counts:
                    tag_counts[base_tag] = 1
                    tag = base_tag
                else:
                    tag_counts[base_tag] += 1
                    tag = f"{base_tag}_{tag_counts[base_tag]}"

                # Store tag and its responses
                self.intents.append(tag)
                self.intents_responses[tag] = intent['responses']

                # Process each pattern into bag-of-words format
                for pattern in intent['patterns']:
                    pattern_words = self.tokenize_and_lemmatize(pattern)
                    self.vocabulary.extend(pattern_words)
                    self.documents.append((pattern_words, tag))

            self.vocabulary = sorted(set(self.vocabulary))


    def prepare_data(self):
        """
        Converts all tokenized patterns into numerical BoW features and labels.
        """
        bags = []
        indices = []

        for document in self.documents:
            words = document[0]
            bag = self.bag_of_words(words)

            intent_index = self.intents.index(document[1])

            bags.append(bag)
            indices.append(intent_index)

        self.X = np.array(bags, dtype=np.uint8)
        self.y = np.array(indices)

    def train_model(self, batch_size, lr, epochs):
        """
        Trains the chatbot model using a feedforward network.

        Args:
            batch_size (int): Training batch size.
            lr (float): Learning rate.
            epochs (int): Number of training epochs.
        """
        X_tensor = torch.tensor(self.X, dtype=torch.float32)
        y_tensor = torch.tensor(self.y, dtype=torch.long)

        dataset = TensorDataset(X_tensor, y_tensor)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        self.model = ChatbotModel(self.X.shape[1], len(self.intents)) 

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=lr)

        for epoch in range(epochs):
            running_loss = 0.0

            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}: Loss: {running_loss / len(loader):.4f}")

    def save_model(self, model_path, dimensions_path):
        """
        Saves the model weights and its architecture details.

        Args:
            model_path (str): File to save model weights.
            dimensions_path (str): File to save input/output sizes.
        """
        torch.save(self.model.state_dict(), model_path)

        with open(dimensions_path, 'w') as f:
            json.dump({
                'input_size': self.X.shape[1],
                'output_size': len(self.intents)
            }, f)

    def load_model(self, model_path, dimensions_path):
        """
        Loads a trained model and its input/output dimensions.

        Args:
            model_path (str): Path to model weights.
            dimensions_path (str): Path to metadata JSON file.
        """
        with open(dimensions_path, 'r') as f:
            dimensions = json.load(f)

        self.model = ChatbotModel(dimensions['input_size'], dimensions['output_size'])
        self.model.load_state_dict(torch.load(model_path))
        self.model.eval()

    def process_message(self, input_message):
        """
        Processes a user message:
        - Converts to BoW vector
        - Predicts intent
        - Returns a random response from predicted tag

        Args:
            input_message (str): User's message.

        Returns:
            str | None: A chatbot response.
        """
        words = self.tokenize_and_lemmatize(input_message)
        bag = self.bag_of_words(words)

        bag_tensor = torch.tensor([bag], dtype=torch.float32)

        self.model.eval()
        with torch.no_grad():
            predictions = self.model(bag_tensor)

        predicted_class_index = torch.argmax(predictions, dim=1).item()
        predicted_intent = self.intents[predicted_class_index]

        responses = self.intents_responses.get(predicted_intent)
        return random.choice(responses) if responses else None


**Run the Chatbot – CLI Inference Loop**

This script loads the trained model and lets the user chat with the assistant in the terminal.
Use `/quit` to exit the conversation.

In [ ]:
if __name__ == '__main__':
    #assistant = ChatbotAssistant('dailydialog_intents.json')
    #assistant.parse_intents()
    #assistant.prepare_data()
    #assistant.train_model(batch_size=8, lr=0.001, epochs=100)

    #assistant.save_model('chatbot_model.pth', 'dimensions.json')

    assistant = ChatbotAssistant('dailydialog_intents.json')
    assistant.parse_intents()
    assistant.load_model('chatbot_model.pth', 'dimensions.json')

    print("🤖 Chatbot is ready! Type your message below (or type /quit to exit):\n")

    # Interactive chat loop
    while True:
        message = input("🗣️ You: ")

        if message.strip().lower() == '/quit':
            print("👋 Chatbot session ended. Goodbye!")
            break

        response = assistant.process_message(message)

        if response:
            print(f"🤖 Bot: {response}")
        else:
            print("🤖 Bot: I'm not sure how to respond to that.")

🤖 Chatbot is ready! Type your message below (or type /quit to exit):

🤖 Bot: I want to study Mandarin and international relations .
🤖 Bot: Thank you . I ’ d better get going . I don ’ t want to be late for lunch . Mom would worry .
🤖 Bot: I ’ Ve been to all of them .
🤖 Bot: Yes , I do . But my characters are very bad .
🤖 Bot: I wish to move up to higher positions with acquisition of more experience in the future .
🤖 Bot: You just said your English needs work , yes ?
